In [ ]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import tables
from tqdm import tqdm
import os
from random import shuffle
from IPython.display import clear_output
import time
%matplotlib inline
import os
import psutil

process = psutil.Process(os.getpid())
process

USE_GPU = True
os.environ['CUDA_VISIBLE_DEVICES'] = '0' if USE_GPU else ''

EPS = 1e-15

In [ ]:
# for phase 1 set num_classes=2
# for phase 2 set num_classes=4
num_classes = 2

In [ ]:
train_file = 'train_1-2.hdf5'
test_file = 'test_1-2.hdf5'
submission_file = 'submission_1-2.hdf5'

In [ ]:
%load_ext autoreload
%autoreload 2

# Prepare data

In [ ]:
from tools.base import plot_3d, hdf5_to_numpy
from tools.tools import calculate_metrics_on_graph, stretch_array, prepare_data_for_submission

In [ ]:
%%time
# N -- number of enties to read. Either int or np.inf. In latter case all entries are readed.
N = 20
X, Y, M, N = hdf5_to_numpy(file=train_file, n=N, num_classes=num_classes)

In [ ]:
k = -1
plot_3d(X[k], Y[k])

# Reduce dimensionality of data with clustering

We have a lot of reduncancy in the data and only four features: x, y, z, E.  One possible way to deal with it is to preprocess data in the following way:

__1)__ Collapse points in small blobs using some criteria. For example, 

    * blob size < 10;
    * max distance in the blob < 1th percentile in full distance matrix.
    
![](img/clusters.png)
    
__2)__ For each blob creacte features that represent it geometrical properties: principal axes and 'sizes' of blob in these directions;

![](img/pca.png)

__3)__ Connect blobs with edges generated by k-nearest neighbour graph algorithm;

__4)__ Train MPNN-model on this data.

__5)__ Predict labels for blobs.

__6)__ Assign blobs labels to all underlying hits.

Parameters:

__max_cl__ -- maximum size of blobs;

__fraction__ -- percentile cut level;

__n_neighbors__ -- number of neighbours for k-nearest neighbours graph algo(http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.kneighbors_graph.html)

__out_degree_max__ -- max number of out-edges for each node;

__in_degree_max__ -- max number of in-edges for each node.

In [ ]:
max_cl = 10
fraction = 1
n_neighbors = 3


in_degree_max, out_degree_max = 0, 0

In [ ]:
from tools.clustering import Cluster, simple_clusterer, clusterise_data
from tools.cluster_aggregation import principal_axes, aggregate_clusters
from tools.graph_construction import generate_edges, generate_graph_dataset

In [ ]:
X_clusters_graph = []
for k in tqdm(range(len(X))):
    if len(X[k]) == 0:
        continue
        
    # clustering data
    X_cluster, Y_cluster, M_cluster = clusterise_data(X[k], Y[k], M[k], verbose=False, 
                                                      max_cl=max_cl, fraction=fraction)
    
    # aggregation of cluster statistics
    X_cluster_condensed = aggregate_clusters(X_cluster, Y_cluster, M_cluster)

    # construction of graph based on aggregated statistics
    X_cluster_graph, in_degree_max_local, out_degree_max_local = generate_graph_dataset(X_cluster_condensed=X_cluster_condensed, 
                                                                                        n_neighbors=n_neighbors)
    in_degree_max = max(in_degree_max_local, in_degree_max)
    out_degree_max = max(out_degree_max_local, out_degree_max)
    
    X_clusters_graph.append(X_cluster_graph)

### What is __in_degree_max__ and __out_degree_max__?

Well, when you are working with tensorflow you have to specify shape of your data(at least number of columns).

```
shape = (number_of_nodes, out_degree/in_degree)
```

__max_out_degree__ is fixed and equal __n_neighbors__ in our setting, but __in_degree_max__ could be different across different events. 

To anticipate it we are padding all events with edges to non-existing node. Latter this should be taken into account in the MPNN-algorithm.

In [ ]:
in_degree_max, out_degree_max

In [ ]:
from tools.tools import calculate_metrics_on_graph, stretch_array

for X_cluster_graph in X_clusters_graph:
    X_cluster_graph['X_cluster_messages_out'] = stretch_array(X_cluster_graph['X_cluster_messages_out'], 
                                                              n=out_degree_max, 
                                                              fill_value=len(X_cluster_graph['X_cluster_edges']))
    
    X_cluster_graph['X_cluster_messages_in'] = stretch_array(X_cluster_graph['X_cluster_messages_in'], 
                                                              n=in_degree_max, 
                                                              fill_value=len(X_cluster_graph['X_cluster_edges']))

## Deep learning model (MPNN)

In [ ]:
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda, GRUCell, GRU
from keras.optimizers import RMSprop
from keras import backend as K
import keras
import tensorflow as tf
from keras.layers import Dropout
from keras.models import Sequential
from keras.activations import relu

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

### Placeholders

In [ ]:
X_cluster_graph['X_cluster_nodes'].shape, X_cluster_graph['X_cluster_edges'].shape

In [ ]:
ndim_features_nodes = 12
ndim_features_edges = 5
ndim_message = 10

In [ ]:
X_nodes = K.placeholder(shape=(None, ndim_features_nodes)) # features of nodes
X_edges = K.placeholder(shape=(None, ndim_features_edges)) # features of edges
X_labels = K.placeholder(shape=(None, num_classes)) # labels

X_nodes_in_out = K.placeholder(shape=(None, 2), dtype=np.int32) # edges
X_messages_in = K.placeholder(shape=(None, in_degree_max), dtype=np.int32) # shape = (none, size of neighbourhood)
X_messages_out = K.placeholder(shape=(None, out_degree_max), dtype=np.int32) # shape = (none, size of neighbourhood)

fake_message_const = K.constant(value=[ndim_message * [-np.inf]]) 
fake_message_const = K.constant(value=[ndim_message * [0]]) 

In [ ]:
placeholders = {
    'X_nodes': X_nodes,
    'X_edges': X_edges,
    'X_labels': X_labels,
    'X_nodes_in_out': X_nodes_in_out,
    'X_messages_in': X_messages_in,
    'X_messages_out': X_messages_out
}

### NNs

In [ ]:
steps = 3

In [ ]:
message_passers = {
    0: Sequential(layers=[
                      Dense(16, input_shape=(2 * ndim_features_nodes + ndim_features_edges,), activation=relu), 
                      Dropout(rate=0.2),
                      Dense(ndim_message, activation=relu),
                      Dropout(rate=0.2),
                  ]
                 ),
    1: Sequential(layers=[
                      Dense(16, input_shape=(2 * ndim_features_nodes + ndim_features_edges,), activation=relu), 
                      Dropout(rate=0.2),
                      Dense(ndim_message, activation=relu),
                      Dropout(rate=0.2),
                  ]
                 ),    
    2: Sequential(layers=[
                      Dense(16, input_shape=(2 * ndim_features_nodes + ndim_features_edges,), activation=relu), 
                      Dropout(rate=0.2),
                      Dense(ndim_message, activation=relu),
                      Dropout(rate=0.2),
                  ]
                 )
}


#state_updater = tf.contrib.rnn.GRUCell(num_units=ndim_features_nodes, )
state_updater = Dense(ndim_features_nodes, input_shape = (2 * ndim_message + ndim_features_nodes,))
readout = Dense(num_classes, input_shape=(ndim_features_nodes,), activation=keras.activations.softmax)

### MPNN construction

A brief explanation of MPNN algorithm in a diagram for a following toy graph:

![](img/example_graph.png)

Algorithm:

![](img/mpnn.png)


And corresponding code with comments:

In [ ]:
def build_network(X_nodes, X_edges, X_nodes_in_out, 
                  X_messages_in, X_messages_out, message_passers, 
                  state_updater, readout, ndim_features_nodes, fake_message_const, steps):
    # nodes 'talks' to each other several times which is defined by __step__ parameter
    for step in range(steps):
        # messages from node to node
        messages = message_passers[step](
            K.concatenate(
                [
                    K.reshape(K.gather(reference=X_nodes, indices=X_nodes_in_out), 
                              shape=(-1, 2 * ndim_features_nodes)), 
                    X_edges
                ], axis=1
            )
        )
        # correct dealing with non-existing edge
        messages = K.concatenate([messages, fake_message_const], axis=0)
        messages = tf.where(tf.is_inf(messages), tf.zeros_like(messages), messages)

        # aggregating messages that came into the node
        messages_aggregated_in = K.max(K.gather(reference=messages, indices=X_messages_in), axis=1)
        # ... and those exiting node
        messages_aggregated_out = K.max(K.gather(reference=messages, indices=X_messages_out), axis=1)

        # update nodes states based on messages and previous state
        X_nodes = state_updater(K.concatenate([messages_aggregated_in, messages_aggregated_out, X_nodes], axis=1))

    return readout(X_nodes)

In [ ]:
from tools.mpnn import build_network, run_train, run_test

In [ ]:
X_predictions = build_network(X_nodes=X_nodes, 
                              X_edges=X_edges, 
                              X_nodes_in_out=X_nodes_in_out, 
                              X_messages_in=X_messages_in, 
                              X_messages_out=X_messages_out, 
                              message_passers=message_passers, 
                              state_updater=state_updater, 
                              readout=readout, 
                              steps=steps, 
                              ndim_features_nodes=ndim_features_nodes,
                              fake_message_const=fake_message_const)

In [ ]:
loss_tf = tf.reduce_mean(keras.losses.categorical_crossentropy(X_labels, X_predictions))
accuracy_tf = tf.reduce_mean(keras.metrics.categorical_accuracy(X_labels, X_predictions))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss_tf, var_list=tf.trainable_variables())

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
init.run(session=sess)

### Split dataset

In [ ]:
TRAIN_SIZE = int(len(X_clusters_graph) * 0.8)

shuffle(X_clusters_graph)

X_clusters_graph_train = X_clusters_graph[:TRAIN_SIZE]
X_clusters_graph_eval = X_clusters_graph[TRAIN_SIZE:]

### Train

In [ ]:
losses = []
accuracies = []
roc_aucs = []

for epoch in tqdm(range(100)):
    loss_float = 0
    accuracy_float = 0
    
    losses_epoch = []
    accuracies_epoch = []
    roc_aucs_epoch = []
    for X_cluster_graph in X_clusters_graph_train:
        predictions, (loss, accuracy) = run_train(X_cluster_graph=X_cluster_graph,
                                   X_predictions=X_predictions,
                                   optimizer=optimizer, sess=sess, 
                                   ndim_features_nodes=ndim_features_nodes, 
                                   ndim_features_edges=ndim_features_edges, 
                                   placeholders=placeholders,
                                   metrics=[loss_tf, accuracy_tf])
        accuracy, roc_auc, predictions_ravel, y_ravel = calculate_metrics_on_graph(X_cluster_graph=X_cluster_graph, predictions=predictions)
        losses_epoch.append(loss)
        accuracies_epoch.append(accuracy)
        roc_aucs_epoch.append(roc_auc)
    clear_output()
    
    roc_aucs.append(np.mean(roc_aucs_epoch))
    plt.plot(roc_aucs)
    plt.show()

    accuracies.append(np.mean(accuracies_epoch))
    plt.plot(accuracies)
    plt.show()

### Eval

In [ ]:
losses_test = []
accuracies_test = []
roc_aucs_test = []
predictions_ravel_total = [] 
y_ravel_total =[]

for X_cluster_graph in X_clusters_graph_eval:
    predictions, (loss, accuracy) = run_test(X_cluster_graph=X_cluster_graph, 
                                              X_predictions=X_predictions,
                                              sess=sess, 
                                              ndim_features_nodes=ndim_features_nodes, 
                                              ndim_features_edges=ndim_features_edges, 
                                              placeholders=placeholders,
                                              metrics=[loss_tf, accuracy_tf])
    X_cluster_graph['predictions'] = predictions
    accuracy, roc_auc, predictions_ravel, y_ravel = calculate_metrics_on_graph(X_cluster_graph=X_cluster_graph, predictions=predictions)
    predictions_ravel_total.append(predictions_ravel)
    y_ravel_total.append(y_ravel)
    losses_test.append(loss)
    accuracies_test.append(accuracy)
    roc_aucs_test.append(roc_auc)

In [ ]:
predictions_ravel_total = np.concatenate(predictions_ravel_total)
y_ravel_total = np.concatenate(y_ravel_total)

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(np.argmax(y_ravel_total, axis=1), np.argmax(predictions_ravel_total, axis=1))
roc_auc = metrics.roc_auc_score(y_ravel_total, predictions_ravel_total)

In [ ]:
accuracy

In [ ]:
roc_auc

## Test

In [ ]:
X_test, Y_test, M_test, N_test = hdf5_to_numpy(file=test_file, n=np.inf, num_classes=num_classes, test=True)

In [ ]:
X_clusters_graph_test = []
for k in tqdm(range(len(X_test))):
    if len(X_test) == 0:
        X_cluster_graph = {}
        X_cluster_graph['empty'] = True
        X_clusters_graph_test.append(X_cluster_graph)
        continue
        
    # clustering
    X_cluster, Y_cluster, M_cluster = clusterise_data(X_test[k], Y_test[k], M_test[k], verbose=False, 
                                                      max_cl=max_cl, fraction=fraction)
    
    # aggregation
    X_cluster_condensed = aggregate_clusters(X_cluster, Y_cluster, M_cluster)

    # graph data
    X_cluster_graph, in_degree_max_local, out_degree_max_local = generate_graph_dataset(X_cluster_condensed=X_cluster_condensed, 
                                                                                        n_neighbors=n_neighbors, 
                                                                                        in_degree_max=in_degree_max, 
                                                                                        out_degree_max=out_degree_max)

    X_cluster_graph['empty'] = False
    X_clusters_graph_test.append(X_cluster_graph)

In [ ]:
def prepare_data_for_submission(X_cluster_graph):
    submission_answer = np.zeros((192, 192, 192))
    shift = 0 if num_classes==4 else 1
    for cluster, prediction in zip(X_cluster_graph['clusters'], X_cluster_graph['predictions']):
        submission_answer[cluster['M'][:, 0], cluster['M'][:, 1], cluster['M'][:, 2]] = np.argmax(prediction) + shift
    X_cluster_graph['submission_answer'] = submission_answer

In [ ]:
for X_cluster_graph in X_clusters_graph_test:
    if not X_cluster_graph['empty']:
        predictions, (loss, accuracy) = run_test(X_cluster_graph=X_cluster_graph, 
                                                  X_predictions=X_predictions,
                                                  sess=sess, 
                                                  ndim_features_nodes=ndim_features_nodes, 
                                                  ndim_features_edges=ndim_features_edges, 
                                                  placeholders=placeholders,
                                                  metrics=[loss_tf, accuracy_tf])
        X_cluster_graph['predictions'] = predictions
    
    prepare_data_for_submission(X_cluster_graph)

In [ ]:
import tables
expectedrows = len(X_test)
FILTERS = tables.Filters(complevel=5, complib='zlib', shuffle=True, bitshuffle=False, fletcher32=False, least_significant_digit=None)
f_submission = tables.open_file(submission_file, 'w', filters=FILTERS)
preds_array = f_submission.create_earray('/', 'pred', tables.UInt32Atom(), (0,192,192,192), expectedrows=expectedrows)

for i in tqdm(range(expectedrows)):
    preds_array.append(np.expand_dims(X_clusters_graph_test[i]['submission_answer'], axis=0))

preds_array.close()
f_submission.close()